In [1]:
import mercury as mr
from IPython.display import HTML
from dotenv import load_dotenv
import html as _html
import os
_ = load_dotenv(".ollama.env") 

In [2]:
import re
import html as _html

def strip_html_fences(text: str) -> str:
    # remove ```html ... ``` or ``` ... ```
    text = re.sub(r"^```(?:html)?\s*", "", text.strip(), flags=re.IGNORECASE)
    text = re.sub(r"^```(?:svg)?\s*", "", text.strip(), flags=re.IGNORECASE)
    text = re.sub(r"\s*```$", "", text.strip())
    return text

def is_html(text: str) -> bool:
    text = text.strip().lower()

    # obvious HTML markers
    if text.startswith("<!doctype html") or text.startswith("<html"):
        return True

    # common HTML tags
    return bool(re.search(
        r"<(div|span|p|h1|h2|h3|table|ul|ol|li|iframe|script|style|body|head)[\s>]",
        text
    ))
def is_svg(text: str) -> bool:
    if not text:
        return False

    text = text.strip().lower()

    # direct svg root
    if text.startswith("<svg"):
        return True

    # svg with xml header or doctype
    if re.search(r"<svg[\s>]", text):
        return True

    return False

In [3]:
from ollama import Client

client = Client(
    host="https://ollama.com",
    headers={'Authorization': 'Bearer ' + os.environ.get('OLLAMA_API_KEY')}
)

In [4]:
messages = [{
    'role': 'system',
    'content': 'Respond with SVG ONLY'
}]

In [5]:
left, right = mr.Columns(2)

ColumnsBox(children=(ColumnOutput(layout=Layout(flex='1 1 0px', min_width='100px'), _dom_classes=('mljar-colum…

In [6]:
with right:
    preview, code = mr.Tabs(["Preview", "Code"])

In [7]:
with left:
    chat = mr.Chat()

In [8]:
prompt = mr.ChatInput()

In [12]:
if prompt.value:
    user_msg = mr.Message(prompt.value)
    chat.add(user_msg)
    messages += [{'role': 'user', 'content': prompt.value}]
    response = client.chat(model="gpt-oss:120b", messages=messages)
    
    messages += [
      {
        'role': response.message.role,
        'content': response.message.content,
      },
    ]
    
    html_code = strip_html_fences(response.message.content)
    if is_html(html_code) or is_svg(html_code):
        with preview:
            preview.clear_output(wait=True)
            escaped = _html.escape(html_code, quote=True)
            display(HTML(f"""
            <iframe
              style="width:100%; height:600px; border:1px solid #ddd; border-radius:8px;"
              srcdoc="{escaped}"
              sandbox="allow-forms allow-modals allow-popups allow-same-origin"
            ></iframe>
            """))
        with code:
            preview.clear_output(wait=True)
            print(html_code)
    else:
        assistant_msg = mr.Message(response.message.content, role="ai", emoji="🎄")
        chat.add(assistant_msg)
        

In [13]:
from mercury.manager import WidgetsManager

In [14]:
WidgetsManager.widgets.keys()

dict_keys(['Columns.86392566.1.9b089310', 'Tabs.af8b53e0.2.7e58ca6c.Preview|Code', 'ChatInput.08f9f624.1.dc9b55c0'])